In [2]:
#Repliación de ejemplo

#Código de ejemplo de WVAM
  
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch

#dataset ciempies

dataset = load_dataset("ciempiess/ciempiess_test")


In [3]:
#modelo predeterminado
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

In [4]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
        num_rows: 3558
    })
})


In [5]:
# Accedemos al subconjunto 'test' del dataset
test_dataset = dataset['test']

<h1>Ejemplo M_07</h1>

In [13]:
# Sample rate deseado (16000 Hz)
sampling_rate = 16000

# Cargamos y filtramos el audio de referencia con el sampling_rate
reference_speaker_id = 'M_07'
reference_audio = test_dataset.filter(lambda x: x["speaker_id"] == reference_speaker_id)["audio"][0]["array"]

# Extraer las características del audio de referencia
reference_input = feature_extractor(reference_audio, sampling_rate=sampling_rate, return_tensors="pt")
reference_embedding = model(**reference_input).embeddings
reference_embedding = torch.nn.functional.normalize(reference_embedding, dim=-1).cpu()

# Funcion para comparar la voz del hablante con los otros hablantes
def compare_speakers(audio1, audio2, threshold=0.86):
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(audio1, audio2)
    return similarity >= threshold

# Lista de los hablantes ya comparados
compared_speakers = []

# Comparamos la voz del hablante M_07 con todos los demas hablantes
for sample in test_dataset:
    if sample["speaker_id"] != reference_speaker_id and sample["speaker_id"] not in compared_speakers:
        other_audio = sample["audio"]["array"]
        
        # Extraemos las caracteristicas del otro hablante asegurando el sampling_rate
        other_input = feature_extractor(other_audio, sampling_rate=sampling_rate, return_tensors="pt")
        other_embedding = model(**other_input).embeddings
        other_embedding = torch.nn.functional.normalize(other_embedding, dim=-1).cpu()

        # Comparación
        if compare_speakers(reference_embedding, other_embedding):
            print(f"La voz del hablante con ID {reference_speaker_id} es similar a la del hablante {sample['speaker_id']}.")
        else:
            print(f"La voz del hablante con ID {reference_speaker_id} NO coincide con la del hablante {sample['speaker_id']}.")
        
        compared_speakers.append(sample["speaker_id"])

La voz del hablante con ID M_07 NO coincide con la del hablante F_01.
La voz del hablante con ID M_07 NO coincide con la del hablante F_05.
La voz del hablante con ID M_07 NO coincide con la del hablante F_07.
La voz del hablante con ID M_07 NO coincide con la del hablante F_04.
La voz del hablante con ID M_07 es similar a la del hablante M_01.
La voz del hablante con ID M_07 NO coincide con la del hablante F_08.
La voz del hablante con ID M_07 es similar a la del hablante M_10.
La voz del hablante con ID M_07 NO coincide con la del hablante M_09.
La voz del hablante con ID M_07 NO coincide con la del hablante M_03.
La voz del hablante con ID M_07 NO coincide con la del hablante M_04.
La voz del hablante con ID M_07 NO coincide con la del hablante F_03.
La voz del hablante con ID M_07 NO coincide con la del hablante F_10.
La voz del hablante con ID M_07 NO coincide con la del hablante F_06.
La voz del hablante con ID M_07 NO coincide con la del hablante F_09.
La voz del hablante con ID

<h1>Ejemplo M_01</h1>

In [12]:
# tasa de muestreo  (16 kHz)
tasa_muestreo = 16000

# hablante de referencia
hablante_objetivo = 'M_01'

# Filtramos el dataset para obtener el audio del hablante objetivo
audio_objetivo = test_dataset.filter(lambda muestra: muestra["speaker_id"] == hablante_objetivo)["audio"][0]["array"]

# Procesamos el audio de referencia para extraer los embeddings
entrada_objetivo = feature_extractor(audio_objetivo, sampling_rate=tasa_muestreo, return_tensors="pt")
embedding_objetivo = model(**entrada_objetivo).embeddings
embedding_objetivo_normalizado = torch.nn.functional.normalize(embedding_objetivo, dim=-1).cpu()

# Función para verificar la similitud de voces
def comparar_voces(embedding1, embedding2, umbral_similitud=0.86):
    similitud_coseno = torch.nn.CosineSimilarity(dim=-1)
    puntuacion_similitud = similitud_coseno(embedding1, embedding2)
    return puntuacion_similitud >= umbral_similitud

# lista de hablantes ya procesados
hablantes_procesados = []

# Recorremos el dataset para comparar el hablante M_01 con los demás
for muestra in test_dataset:
    hablante_actual = muestra["speaker_id"]

    # Comparamos solo con hablantes que no sean el objetivo y que no hayan sido procesados antes
    if hablante_actual != hablante_objetivo and hablante_actual not in hablantes_procesados:
        audio_actual = muestra["audio"]["array"]

        # Procesamiento el audio actual para extraer los embeddings
        entrada_actual = feature_extractor(audio_actual, sampling_rate=tasa_muestreo, return_tensors="pt")
        embedding_actual = model(**entrada_actual).embeddings
        embedding_actual_normalizado = torch.nn.functional.normalize(embedding_actual, dim=-1).cpu()

        # Comparación de voces
        if comparar_voces(embedding_objetivo_normalizado, embedding_actual_normalizado):
            print(f"La voz del hablante {hablante_objetivo} es similar a la del hablante {hablante_actual}.")
        else:
            print(f"La voz del hablante {hablante_objetivo} NO coincide con la del hablante {hablante_actual}.")

        # Marcamos al hablante actual como procesado
        hablantes_procesados.append(hablante_actual)


La voz del hablante M_01 es similar a la del hablante M_07.
La voz del hablante M_01 NO coincide con la del hablante F_01.
La voz del hablante M_01 NO coincide con la del hablante F_05.
La voz del hablante M_01 NO coincide con la del hablante F_07.
La voz del hablante M_01 NO coincide con la del hablante F_04.
La voz del hablante M_01 es similar a la del hablante F_08.
La voz del hablante M_01 es similar a la del hablante M_10.
La voz del hablante M_01 es similar a la del hablante M_09.
La voz del hablante M_01 es similar a la del hablante M_03.
La voz del hablante M_01 NO coincide con la del hablante M_04.
La voz del hablante M_01 NO coincide con la del hablante F_03.
La voz del hablante M_01 NO coincide con la del hablante F_10.
La voz del hablante M_01 NO coincide con la del hablante F_06.
La voz del hablante M_01 NO coincide con la del hablante F_09.
La voz del hablante M_01 es similar a la del hablante M_02.
La voz del hablante M_01 es similar a la del hablante M_06.
La voz del ha

In [2]:
#Repliación de ejemplo

#Código de ejemplo de WVAM
  
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch

#dataset ciempies

#dataset = load_dataset("ciempiess/ciempiess_test")


In [3]:
dataset = load_dataset("ciempiess/ciempiess_light")

In [4]:
#modelo predeterminado
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
        num_rows: 16663
    })
})


In [6]:
# Accedemos al subconjunto 'test' del dataset
test_dataset = dataset['train']

In [9]:
# Sample rate deseado (16000 Hz)
sampling_rate = 16000

# Cargamos y filtramos el audio de referencia con el sampling_rate
reference_speaker_id = 'M_07'
reference_audio = test_dataset.filter(lambda x: x["speaker_id"] == reference_speaker_id)["audio"][0]["array"]

# Extraer las características del audio de referencia
reference_input = feature_extractor(reference_audio, sampling_rate=sampling_rate, return_tensors="pt")
reference_embedding = model(**reference_input).embeddings
reference_embedding = torch.nn.functional.normalize(reference_embedding, dim=-1).cpu()

# Funcion para comparar la voz del hablante con los otros hablantes
def compare_speakers(audio1, audio2, threshold=0.86):
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(audio1, audio2)
    return similarity >= threshold

# Lista de los hablantes ya comparados
compared_speakers = []

# Comparamos la voz del hablante M_07 con todos los demas hablantes
for sample in test_dataset:
    if sample["speaker_id"] != reference_speaker_id and sample["speaker_id"] not in compared_speakers:
        other_audio = sample["audio"]["array"]
        
        # Extraemos las caracteristicas del otro hablante asegurando el sampling_rate
        other_input = feature_extractor(other_audio, sampling_rate=sampling_rate, return_tensors="pt")
        other_embedding = model(**other_input).embeddings
        other_embedding = torch.nn.functional.normalize(other_embedding, dim=-1).cpu()

        # Comparación
        if compare_speakers(reference_embedding, other_embedding):
            print(f"La voz del hablante con ID {reference_speaker_id} es similar a la del hablante {sample['speaker_id']}.")
        else:
            print(f"La voz del hablante con ID {reference_speaker_id} NO coincide con la del hablante {sample['speaker_id']}.")
        
        compared_speakers.append(sample["speaker_id"])

c:\Users\anton\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


La voz del hablante con ID M_07 NO coincide con la del hablante F_32.
La voz del hablante con ID M_07 NO coincide con la del hablante F_20.
La voz del hablante con ID M_07 NO coincide con la del hablante F_05.
La voz del hablante con ID M_07 NO coincide con la del hablante F_09.
La voz del hablante con ID M_07 NO coincide con la del hablante F_12.
La voz del hablante con ID M_07 NO coincide con la del hablante F_25.
La voz del hablante con ID M_07 NO coincide con la del hablante F_28.
La voz del hablante con ID M_07 NO coincide con la del hablante F_07.
La voz del hablante con ID M_07 NO coincide con la del hablante F_14.
La voz del hablante con ID M_07 NO coincide con la del hablante F_18.
La voz del hablante con ID M_07 NO coincide con la del hablante F_01.
La voz del hablante con ID M_07 NO coincide con la del hablante F_33.
La voz del hablante con ID M_07 NO coincide con la del hablante F_08.
La voz del hablante con ID M_07 NO coincide con la del hablante F_29.
La voz del hablante 

In [7]:
import torch

# Sample rate deseado (16000 Hz)
sampling_rate = 16000

# Crear un diccionario para almacenar las embeddings de cada hablante y su conteo de similitudes
embeddings_dict = {}
similarity_counts = {}

# Extraer los IDs de los hablantes del dataset
speaker_ids = list(set([sample["speaker_id"] for sample in test_dataset]))

# Extraer las características de cada hablante y calcular las embeddings
for speaker_id in speaker_ids:
    speaker_audio = test_dataset.filter(lambda x: x["speaker_id"] == speaker_id)["audio"][0]["array"]
    speaker_input = feature_extractor(speaker_audio, sampling_rate=sampling_rate, return_tensors="pt")
    speaker_embedding = model(**speaker_input).embeddings
    speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=-1).cpu()
    embeddings_dict[speaker_id] = speaker_embedding
    similarity_counts[speaker_id] = 0

# Umbral de similitud
threshold = 0.86

# Crear la función de similitud de coseno
cosine_sim = torch.nn.CosineSimilarity(dim=-1)

# Realizar las comparaciones de todas las voces contra todas
for i, speaker_id1 in enumerate(speaker_ids):
    for j, speaker_id2 in enumerate(speaker_ids):
        # Evitar compararse consigo mismoA
        if speaker_id1 != speaker_id2:
            similarity = cosine_sim(embeddings_dict[speaker_id1], embeddings_dict[speaker_id2])
            
            # Si la similitud es mayor o igual al umbral, incrementar el conteo de similitudes del hablante
            if similarity >= threshold:
                similarity_counts[speaker_id1] += 1

# Mostrar el resumen de las similitudes
for speaker_id, count in similarity_counts.items():
    print(f"{speaker_id} tiene similitud con {count} hablantes")


Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

c:\Users\anton\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16663 [00:00<?, ? examples/s]

MemoryError: Unable to allocate 696. KiB for an array with shape (89096,) and data type float64